In [ ]:
# import getpass
# import os

# password = getpass.getpass()
# command = "sudo -S bash trust_apple_corp_root_cas.sh"
# os.system('echo %s | %s' % (password, command))
# os.system('echo %s' % (command))

# %pip install --upgrade geopandas
# %pip install --upgrade matplotlib
# %pip install --upgrade palettable
# %pip install --upgrade sidecar
# %pip install --upgrade lonboard
# %pip install --upgrade pyogrio
# %pip install --upgrade scikit-learn

In [ ]:
import geopandas as gpd
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
from palettable.colorbrewer.diverging import BrBG_10
from sidecar import Sidecar

from lonboard import Map, ScatterplotLayer
from lonboard.colormap import apply_continuous_cmap
# %matplotlib inline

import warnings

# Suppress FutureWarning messages
warnings.simplefilter(action='ignore', category=FutureWarning)

# Global variable to store the currently displayed Sidecar widget
current_sidecar = None

In [ ]:
csv_file = "1900-2018-GH.csv"

gdf = gpd.read_file(
    csv_file,
    engine="pyogrio",
    use_arrow=True,
    X_POSSIBLE_NAMES="longitude",
    Y_POSSIBLE_NAMES="latitude",
    KEEP_GEOM_COLUMNS="NO",
)

## Quick look at the data

In [ ]:
gdf

In [ ]:
gdf.info()

## Check for numeric None, NaN, or NaT values

In [ ]:
def check_numeric_nulls():
    numeric_null_mask = gdf.applymap(lambda x: pd.isna(x))

    # Count the number of numeric null values in each column
    return numeric_null_mask.sum()

check_numeric_nulls()

In [ ]:
# Check for 'None' string cells in the DataFrame
def check_none_string():
    none_string_mask = gdf.applymap(lambda x: x == 'None')

    # Count the number of 'None' string cells in each column
    return none_string_mask.sum()

check_none_string()

In [ ]:
# Check for empty cells in the DataFrame
def check_empty_cells():
    empty_cells_mask = gdf.applymap(lambda x: x == '')

    # Count the number of empty cells in each column
    return empty_cells_mask.sum()

check_empty_cells()

## Convert required columns to numeric and replace NaN values with the median

In [ ]:
# Convert these columns to numeric, coercing errors to NaN
gdf['depth'] = pd.to_numeric(gdf['depth'], errors='coerce')
gdf['mag'] = pd.to_numeric(gdf['mag'], errors='coerce')
# Convert datetime strings to timestamps
# gdf['time'] = pd.to_datetime(gdf['time']).astype(int) / 10**9

# Calculate the median of the 'depth' column
depth_median = gdf['depth'].median()

# Calculate the median of the 'mag' column
depth_median = gdf['mag'].median()

# Fill NaN values with the median
gdf['depth'] = gdf['depth'].fillna(depth_median)
gdf['mag'] = gdf['mag'].fillna(depth_median)

In [ ]:
gdf.info()

In [ ]:
check_empty_cells()

In [ ]:
gdf

In [ ]:
gdf.describe()

In [ ]:
gdf['mag'].hist()

In [ ]:
layer = ScatterplotLayer.from_geopandas(gdf)
map_ = Map(layers=[layer], _height=1800)

# from shapely.geometry import MultiPoint

# #Calculate centroids
# centroids = gdf.centroid

# #Extract centroid coordinates into a MultiPoint object
# multi_point = MultiPoint([geom.xy for geom in centroids])

# # Calculate the median centroid
# median_centroid = multi_point.centroid

# print(f"{median_centroid.coords.xy[0][0]}, {median_centroid.coords.xy[1][0]}")

# map_._initial_view_state={
#     "longitude": median_centroid.coords.xy[0][0], 
#     "latitude": median_centroid.coords.xy[1][0], 
#     "zoom": 7}

# ---------------------------------------------------------------------
# Calculate the centroid for each geometry and collect them
centroids = gdf.geometry.apply(lambda geom: (geom.centroid.x, geom.centroid.y)).tolist()

# Calculate the median centroid
median_centroid = (
    sorted(centroids, key=lambda x: x[0])[len(centroids) // 2][0],  # Median x-coordinate
    sorted(centroids, key=lambda x: x[1])[len(centroids) // 2][1]   # Median y-coordinate
)

# Reverse the order of coordinates to match (latitude, longitude)
# median_centroid = median_centroid[::-1]
# median_centroid

map_._initial_view_state={"longitude": median_centroid[0], "latitude": median_centroid[1], "zoom": 7.5}

In [ ]:
# Function to create and display a Sidecar widget
def display_sidecar():
    global current_sidecar
    if current_sidecar is not None:
        current_sidecar.close()  # Close the currently displayed Sidecar widget
    current_sidecar = Sidecar(title='CA Map')
    with current_sidecar:
        display(map_)  # Replace this with your desired content

# Test the function
display_sidecar()

# with Sidecar(title="CA Map") as sc:
#     display(map_)

In [ ]:
colname = "mag"
gdf[colname] = pd.to_numeric(gdf[colname])

### Linear normalization process to convert magnitude values ranging from 0 to 9 to a range of 0 to 1 to colormap the data

In [ ]:
normalizer = mpl.colors.Normalize(0, 9)
normalized_values = normalizer(gdf[colname])

In [ ]:
normalized_values

In [ ]:
BrBG_10.mpl_colormap

In [ ]:
layer.radius_min_pixels = 0.5
layer.get_fill_color = apply_continuous_cmap(normalized_values, BrBG_10)

In [ ]:
log_normalizer = mpl.colors.LogNorm(3, 9, clip=True)
log_values = log_normalizer(gdf[colname])

In [ ]:
layer.get_fill_color = apply_continuous_cmap(log_values, BrBG_10)
layer.get_radius = log_values * 2500
layer.radius_units = "meters"

In [ ]:
# Extract x and y coordinates from the geometry column
gdf['longitude'] = gdf.geometry.x
gdf['latitude'] = gdf.geometry.y
# gdf

In [ ]:
gdf.plot(kind="scatter", x="longitude", y="latitude", alpha=0.4, s=gdf["mag"], label="magnitude", figsize=(10,7), c="mag", cmap=plt.get_cmap("seismic"), colorbar=True,
    sharex=False)

In [ ]:
from sklearn.model_selection import train_test_split

columns_to_drop = ['time', 'mag', 'id', 'place', 'geohash', 'geometry']
X = gdf.drop(columns_to_drop, axis=1)
y = gdf['mag']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()

# Train the model
model.fit(X_train, y_train)
# model.fit(gdf[["time", "depth", "longitude", "latitude"]], gdf["mag"])

# Make predictions
y_pred = model.predict(X_test)

y_pred

In [ ]:
coef = model.coef_
coef

In [ ]:
intercept = model.intercept_
intercept